In [1]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Get List of Reports Available

In [2]:
import os
from glob import glob
import src.ExportProvider.IBRK.Extract as ibrk
from lxml import etree
import src.ReportingStrategies.Slovenia.ReportGeneration as sir
import src.ReportingStrategies.Slovenia.Mappers.IBRK as ibrkMapper
import src.Common.Configuration as conf
from arrow import Arrow
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "BrokerExports")
generatedDirectory = os.path.join(rootOfProject, "GeneratedReports")

brokerExports = glob("*.xml", root_dir=reportsDirectory)
print(brokerExports)


def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractCashTransactionFromXML(root)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeCashTransactions(brokerReports)

taxPayerInfo = conf.TaxPayerInfo(
    taxNumber = "davcna",
    taxpayerType = conf.TaxPayerType.PHYSICAL_SUBJECT,
    name = "ime priimek",
    address1 = "naslov",
    address2 = None,
    city = "mesto",
    postNumber = "postna #",
    postName = "posta",
    municipalityName = "obcina",
    birthDate = Arrow.now(),
    maticnaStevilka = "emsa",
    invalidskoPodjetje = False,
    resident = True,
    activityCode = "",
    activityName = "",
    countryID = "SI",
    countryName = "Slovenia"
)
basicReport = sir.EDavkiReportWrapper(taxPayerInfo)
envelope = basicReport.createReportEnvelope()


reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"), ReportType=sir.EDavkiDocumentWorkflowType.SELF_REPORT)
dividendReport = sir.EDavkiDividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)

convertedCommonFormat = ibrkMapper.getGenericDividendLineFromIBRKCashTransactions(mergedReports)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)
csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)

xmlReport = dividendReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)



['Tax_Relevant_Export_2022.xml']


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=&crumb=VsRrGkhWw2x


Failed online lookup of ISIN (US9663875089), falling back to hardcoded company definitions
Failed online lookup of ISIN (US9663875089), falling back to hardcoded company definitions


### Stock Trades

In [3]:



def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractTradesFromXML(root)
    return transactions

reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"), ReportType=sir.EDavkiDocumentWorkflowType.SELF_REPORT)

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeTrades(brokerReports)

convertedCommonFormat = ibrkMapper.getGenericTradeLinesFromIBRKTrades(mergedReports)

# print(convertedCommonFormat)
basicReport = sir.EDavkiReportWrapper(taxPayerInfo)
envelope = basicReport.createReportEnvelope()

tradeReport = sir.EDavkiTradesReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)

tradeCsv = tradeReport.generateDataFrameReport(convertedCommonFormat)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-trades.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_KDVP_9.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)
